In [1]:
import pandas as pd 
import yfinance as yf 
import numpy as np
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']
tickers.head()

0     MMM
1     AOS
2     ABT
3    ABBV
4     ACN
Name: Symbol, dtype: object

In [3]:
tickers = tickers.to_list()
price_df = yf.download(tickers,start='2010-01-01')

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


In [4]:
#slice dataframe for target asset 
price_df[price_df.columns[price_df.columns.get_level_values(1) == ('AAPL')]]

,Adj Close,Close,High,Low,Open,Volume
,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2010-01-04,6.505280,7.643214,7.660714,7.585000,7.622500,493729600
2010-01-05,6.516526,7.656429,7.699643,7.616071,7.664286,601904800
2010-01-06,6.412875,7.534643,7.686786,7.526786,7.656429,552160000
2010-01-07,6.401017,7.520714,7.571429,7.466071,7.562500,477131200
2010-01-08,6.443574,7.570714,7.571429,7.466429,7.510714,447610800
...,...,...,...,...,...,...
2023-04-19,167.630005,167.630005,168.160004,165.539993,165.800003,47720200
2023-04-20,166.649994,166.649994,167.869995,165.559998,166.089996,52456400


In [5]:
def slice_df(symbol):
  sliced = price_df.copy()
  sliced = sliced[sliced.columns[sliced.columns.get_level_values(1) == (symbol)]]
  sliced.columns = sliced.columns.droplevel(1)
  sliced.loc[:,'price'] = sliced.Open.shift(-1)
  # sliced['price'] = sliced.Open.shift(-1)
  return sliced[:-1] 

In [6]:
test = slice_df('AAPL')

In [7]:
# Moving Average function
def ma_calc(df,n,m):
  df['sma_1'] = df.Close.rolling(n).mean()
  df['sma_2'] = df.Close.rolling(m).mean()

In [8]:
test

,Adj Close,Close,High,Low,Open,Volume,price
Date,,,,,,,
2010-01-04,6.505280,7.643214,7.660714,7.585000,7.622500,493729600,7.664286
2010-01-05,6.516526,7.656429,7.699643,7.616071,7.664286,601904800,7.656429
2010-01-06,6.412875,7.534643,7.686786,7.526786,7.656429,552160000,7.562500
2010-01-07,6.401017,7.520714,7.571429,7.466071,7.562500,477131200,7.510714
2010-01-08,6.443574,7.570714,7.571429,7.466429,7.510714,447610800,7.600000
...,...,...,...,...,...,...,...
2023-04-18,166.470001,166.470001,167.410004,165.649994,166.100006,49923000,165.800003
2023-04-19,167.630005,167.630005,168.160004,165.539993,165.800003,47720200,166.089996
2023-04-20,166.649994,166.649994,167.869995,165.559998,166.089996,52456400,165.050003


In [9]:
def backtest(df,n,m):
  # df = df.dropna()
  ma_calc(df,n,m)
  in_position = False
  profits = []

  for index,row in df.iterrows():
    if not in_position:
      if (row.sma_1 > row.sma_2) and row.price != 0:
        buyprice = row.price
        in_position = True
    if in_position:
      if (row.sma_1 < row.sma_2):
        profit = (row.price - buyprice)/buyprice
        profits.append(profit)
        in_positon = False
  #my_list = np.array(profits, dtype=np.float64)
  # gain = np.exp(np.log(1+pd.Series(profits)).cumsum())-1
  gain = (pd.Series(profits)+1).prod()
  return gain
#   pr = [round(x,4) for x in profits]
#   return pr

In [10]:
alt = backtest(test,50,100)
alt

inf